In [2]:
import json
from pathlib import Path

In [3]:
# 需要用户输入的内容
path = './明日方舟'
header = '显示名'
meta = {
    "name":"明日方舟对话框",
    "cover":"@/cover.png",
    "describe":"明日方舟的极简风格，没有多加东西，请以实际情况调整。",
    "author":"GULILI",
    "licence":"不可以用于商业用途，无需注明作者"
}

# 创建智能模板

In [5]:
# 载入rgpj
workpath = Path(path)
find_result = list(workpath.glob('*.rgpj'))
if find_result:
    rplgen_project = json.load(open(find_result[0].absolute()))
# 创建一个空白的模板
rplgen_intel_template = {}
# 写入meta信息
# 封面要求：540p，jpg
rplgen_intel_template['meta'] = meta
# 项目配置，移除原有的项目名称和封面
config = rplgen_project['config'].copy()
config.pop('Name')
config.pop('Cover')
rplgen_intel_template['config'] = config

# 排序媒体
media_order = {
    'Pos'       : ['Pos','FreePos','PosGrid','BezierCurve'],
    'Text'      : ['Text', 'StrokeText', 'RichText', 'HPLabel'],
    'Animation' : ['Animation'],
    'Bubble'    : ['Bubble','Balloon','DynamicBubble','ChatWindow'],
    'Background': ['Background'],
    'Audio'     : ['Audio','BGM'],
}
static_media = {}
for type in media_order:
    this_type:list = media_order[type]
    for name in rplgen_project['mediadef']:
        item = rplgen_project['mediadef'][name]
        if item['type'] in this_type:
            static_media[name] = item
# 添加媒体
rplgen_intel_template['media'] = {
    'static':static_media,
    'dynamic':{}
}
# 检视角色
# 获取自定义列的信息
table_col = ['Name','Subtype','Animation','Bubble','Voice','SpeechRate','PitchRate']
custom = []
static_list = []
dynamic_list = []
for keyword in rplgen_project['chartab']:
    this_char = rplgen_project['chartab'][keyword]
    # 检视自定义
    for col in this_char:
        if col not in table_col and col not in custom:
            custom.append(col)
    # 检视名字
    name:str = this_char['Name']
    if name.isdigit():
        dynamic_list.append(keyword)
    else:
        static_list.append(keyword)
dynamic_charactor = {}
static_charactor = {}
# 静态角色采用的列
static_use_col = ['Name','Subtype','Animation','Bubble'] + custom
# 动态角色采用的列，不含header
dynamic_use_col = ['Animation','Bubble'] + custom
dynamic_use_col.remove(header)
# 获取静态角色
for keyword in static_list:
    this_char = rplgen_project['chartab'][keyword]
    tplt_char = {}
    name = this_char['Name']
    for col in static_use_col:
        tplt_char[col] = this_char[col]
    static_charactor[name] = tplt_char
# 获取动态角色
for keyword in dynamic_list:
    this_char = rplgen_project['chartab'][keyword]
    tplt_char = {}
    name = this_char['Name']
    for col in dynamic_use_col:
        tplt_char[col] = this_char[col]
    dynamic_charactor[name] = tplt_char
# 添加角色
rplgen_intel_template['charactor'] = {
    "custom":custom,
    "header":header,
    'static':static_charactor,
    'dynamic':dynamic_charactor
}
# 剧本
rplgen_intel_template['preset'] = rplgen_project['logfile']['preset']
if 'example' in rplgen_project['logfile']:
    rplgen_intel_template['example'] = rplgen_project['logfile']['example']
else:
    rplgen_intel_template['example'] = None
# 输出
with open(workpath.absolute()/'main.rgint','w',encoding='utf-8') as rgint:
    json.dump(rplgen_intel_template,rgint,ensure_ascii=False,indent=4)

# 使用steamwork API

In [1]:
import sys,os
if sys.version_info >= (3, 8):
  os.add_dll_directory(os.getcwd()) # Required since Python 3.8
from steamworks import STEAMWORKS # Import main STEAMWORKS class

In [10]:
# 初始化steamwork类
steamworks = STEAMWORKS()
steamworks.initialize()

d:\GitHub\RplGen-Gallery.github.io\SteamworksPy64.dll


OSError: [WinError 127] 找不到指定的程序。

In [11]:
from ctypes import CDLL

In [13]:
CDLL('./SteamworksPy64.dll')

FileNotFoundError: Could not find module 'd:\GitHub\RplGen-Gallery.github.io\SteamworksPy64.dll' (or one of its dependencies). Try using the full path with constructor syntax.

In [ ]:
# 初始化steamwork类
steamworks = STEAMWORKS()
steamworks.initialize()

d:\GitHub\RplGen-Gallery.github.io\SteamworksPy64.dll


OSError: [WinError 127] 找不到指定的程序。

# 上传到创意工坊

In [6]:
import json

In [7]:
path = './重返未来'
version = 'RGS @ beta 2.0.6'

In [8]:
vdf_tplt = open('./workshop_newitem.vdf','r').read()
meta = json.load(open(path+'/main.rgint','r'))['meta']
name = meta['name']

folder = Path(path).absolute().__str__()
cover = meta['cover'].replace('@',folder).replace('/','\\')

description = '简介：{describe} 作者：{author} 授权：{licence}'.format(**meta)
# 保存vdf
with open(f'./steamcmd/{name}.vdf','w') as of:
    of.write(
        vdf_tplt
        .replace('{path}',folder)
        .replace('{cover}',cover)
        .replace('{title}',meta['name'])
        .replace('{description}',description)
        .replace('{version}',version)
    )

In [ ]:
if sys.version_info >= (3, 8):
  os.add_dll_directory(os.getcwd()) # Required since Python 3.8

## steamcmd
```bash
cd ./steamcmd
steamcmd.exe +login $myLoginName $myPassword +workshop_build_item ${tplt_name}.vdf +quit'
```